# Thoughts on quantile normalization

Quantile normalization is a method used to make the distribution of gene expression levels the same across all samples, making them comparable.

1. **Rank the genes** in each sample from lowest to highest expression.
2. **Calculate the average** expression value for genes that share the same rank across all samples.
3. **Replace the original values** with these average values for each corresponding rank.
4. **Reorder the genes** in each sample back to their original order.

`shears` takes as input a single cell matrix and a bulk matrix with raw counts that need to be normalized. These input matrices are subsetted for highly variable genes (hvg=2000) computed on the single cell data to remove background noise.

For quantile normalisation `scissor` first [combines](https://github.com/sunduanchen/Scissor/blob/311560a1f160f665917e7da0fcf289914ee99773/R/Scissor.R#L78-L79) the single cell and bulk data before spliting it back after the transformation ([see](#3a-quantile-normalize-scissor-approach)). To me, this does not make much sense, as I would expect the bulk samples to be more similar to one another than to the individual single cells using hvgs. Especially for large single cell matrices (>1M cells) - like the CRC-atlas - I would expect `sklearn.preprocessing.quantile_transform(X)` with the default of `n_quantiles=1000` to put all bulk patients in the same quantile.

The `class-specific` strategy is an alternative approach that first splits the data by sample class-labels before performing quantile normalization independently on each split [PMID: 32968196](https://www.nature.com/articles/s41598-020-72664-6). The `discrete` strategy takes the `Class-specific` approach further, and also accounts for the batch factor. Each split (by class and batch) are then quantile-normalized separately, and then recombined into one dataset.

For the CRC-atlas, I would expect the biggest technical batch effects between platforms/datasets. When subsetting for hvgs, enriched datasets will have low counts for the "missing" cells, while bulk samples will have counts across all cell types. `class-specific` effects could be across cell types. (what resolution? fine/coarse). Within a given cell type I am not sure if the same genes have the highest counts across platforms/datasets, which would put them into the same quantile regardless of the absolute value (i.e smartseq samples will have much higher counts than 10x samples, but is the relative distribution from low to high within cell types the same?).

### 1. Load data

1. **CRC single cell atlas:** sc dataset
2. **TCGA-COAD/READ cohort:** bulk dataset with clinical annotations
3. **AC-ICAM cohort:** bulk dataset with more accurate clinical annotations; [PMID: 37202560](https://www.nature.com/articles/s41591-023-02324-5) 348 patients with primary colon cancer. This is the recently published follow up to the TCGA-COAD/READ cohort that was mostly used for colorectal cancer up to this point

In [1]:
import os

import anndata
import numpy as np
import pandas as pd
import scanpy as sc
import shears as sh
from nxfvars import nxfvars
from threadpoolctl import threadpool_limits

In [2]:
cpus = nxfvars.get("cpus", 16)
os.environ["NUMBA_NUM_THREADS"] = str(cpus)
threadpool_limits(cpus)
sc.settings.n_jobs = cpus

In [3]:
adata_path = nxfvars.get(
    "adata_path",
    "/data/scratch/marteau/results_old/crc-atlas/uncompressed_h5ad/core_atlas-adata.h5ad",
)
AC_ICAM_bulk_path = nxfvars.get(
    "AC_ICAM_bulk_path",
    "/data/scratch/marteau/results/crc-atlas/downstream_analyses/shears/AC_ICAM_bulk-adata.h5ad",
)
TCGA_bulk_path = nxfvars.get(
    "TCGA_bulk_path",
    "/data/scratch/marteau/results/crc-atlas/downstream_analyses/shears/TCGA_bulk_counts-adata.h5ad",
)
artifact_dir = nxfvars.get(
    "artifact_dir",
    "/data/scratch/marteau/results/crc-atlas/downstream_analyses/shears/",
)

In [4]:
# The adatas all contain raw counts data
adata_sc = sc.read_h5ad(adata_path)
adata_icam = sc.read_h5ad(AC_ICAM_bulk_path)
adata_tcga = sc.read_h5ad(TCGA_bulk_path)

In [5]:
adata_sc.shape

(4264929, 28476)

In [6]:
adata_tcga.shape

(622, 60483)

In [7]:
adata_icam.shape

(348, 58395)

In [8]:
# Subset for primary tumor samples -> same sample type as bulk
adata_sc = adata_sc[adata_sc.obs["sample_type"].isin(["tumor"])].copy()

Impact of further subsetting? For example 

* removing sorted datasets
* keep only core/border samples
* keep only MSS/MSI samples

When to subset? before hvg calculation or after?

In [9]:
# Replace normalized counts and remove duplicate layer
adata_sc.X = adata_sc.layers["counts"].copy()
del adata_sc.layers["counts"]

### 2. Subset for highly variable genes

Why 2000 hvgs? What is the ideal number?

In [10]:
 # Filter raw bulk matrices for informative genes
sc.pp.filter_genes(adata_icam, min_counts=20)
sc.pp.filter_genes(adata_icam, min_cells=20)  # Expression > 20 counts in min 20 patients

sc.pp.filter_genes(adata_tcga, min_counts=20)
sc.pp.filter_genes(adata_tcga, min_cells=20)

In [11]:
adata_tcga.shape

(622, 46585)

In [12]:
adata_icam.shape

(348, 37140)

In [13]:
# Get common genes across cohorts (ideally using ensembl ids)
adata_hvg = adata_sc[
    :,
    (adata_sc.var_names.isin(adata_tcga.var_names))
    & adata_sc.var_names.isin(adata_icam.var_names),
].copy()
adata_hvg.shape

(1846319, 24810)

In [14]:
# To find a meaningfull signal in the bulk sample I would expect a minimum number of cells to be present
sc.pp.filter_genes(adata_hvg, min_cells=200)

In [15]:
# %%
# Remove few samples in atlas that produce "ValueError: b'There are other near singularities as well." -> Use a loop calling sc.pp.highly_variable_genes on individual samples to get a list of failling ones
hvg = sc.pp.highly_variable_genes(
    adata_hvg[
        ~adata_hvg.obs["sample_id"].isin(
            [
                "Joanito_2022_SG1.MUX9380",
                "Khaliq_2022.T_CAC5",
                "Li_2017.CRC03_tumor",
                "Li_2017.CRC06_tumor",
                "Li_2017.CRC08_tumor",
                "Pelka_2021_10Xv2.C138_T_0_0_0_c2_v2",
                "Pelka_2021_10Xv2_CD45Pos.C138_T_0_0_1_c1_v2",
                "Pelka_2021_10Xv2_CD45Pos.C138_T_0_0_1_c2_v2",
                "Wu_2022_CD45Pos.P1_Colon_T",
                "Wu_2022_CD45Pos.P15_Colon_T",
                "Wu_2022_CD45Pos.P20_Colon_T",
                "Wu_2024_CD66bPos.COAD2_T",
                "Wu_2024_CD66bPos.COAD3_T",
                "Wu_2024_CD66bPos.COAD14_T",
                "Wu_2024_CD66bPos.COAD15_T",
                "Wu_2024_CD66bPos.COAD16_T",
                "Wu_2024_CD66bPos.COAD19_T",
                "Wu_2024_CD66bPos.COAD20_T",
                "Zhang_2020_10X_CD45Pos.T_P0123",
                "Zhang_2020_10X_CD45Pos.T_P0305",
                "deVries_2023_LUMC.HTO1",
                "deVries_2023_LUMC.HTO6",
            ]
        )
    ],
    n_top_genes=2000,
    subset=False,
    flavor="seurat_v3",
    # layer="counts",
    batch_key="sample_id",
    span=0.3,  # does not work with default span=0.3 for above samples (probably not enough cells)
    inplace=False,
)

In [16]:
hvg_dict = hvg["highly_variable"].to_dict()
adata_sc.var["highly_variable"] = adata_sc.var_names.map(hvg_dict).fillna(value=False)

In [17]:
# We can have a look at the hvg classes -> It would be possible to exclude classes
adata_sc[:, adata_sc.var["highly_variable"]].var["Class"].value_counts()

Class
protein_coding                    1541
lncRNA                             200
IG_V_gene                          104
TR_V_gene                           81
IG_V_pseudogene                     18
IG_C_gene                           12
TR_J_gene                           10
miRNA                               10
transcribed_unitary_pseudogene       7
IG_J_gene                            4
IG_C_pseudogene                      3
TR_C_gene                            3
TR_V_pseudogene                      3
Mt_tRNA                              2
IG_J_pseudogene                      1
artifact                             1
Name: count, dtype: int64

In [18]:
# Subset for hvgs
adata_sc = adata_sc[:, adata_sc.var["highly_variable"]].copy()

In [19]:
# Subset bulk for atlas hvg
adata_icam = adata_icam[:, adata_icam.var_names.isin(adata_sc.var_names)].copy()
# Subset bulk for atlas hvg
adata_tcga = adata_tcga[:, adata_tcga.var_names.isin(adata_sc.var_names)].copy()

In [20]:
adata_sc.shape

(1846319, 2000)

In [21]:
adata_icam.shape

(348, 2000)

In [22]:
adata_tcga.shape

(622, 2000)

### 3. Quantile normalize
How to handle already normalized counts?

In [23]:
import sklearn.preprocessing
from anndata import AnnData


def quantile_norm(adata: AnnData, *, layer=None, key_added="quantile_norm", **kwargs):
    """Perform quantile normalization on AnnData object

    Stores the normalized data in a new layer with the key `key_added`.
    """
    X = adata.X if layer is None else adata.layers[layer]
    adata.layers[key_added] = sklearn.preprocessing.quantile_transform(X, **kwargs)

In [24]:
# As the bulk data has less then 1000 patients using the default n_quantiles=1000 will put each patient in his own quantile. Not sure I want this. Id rather have similar patients in the same quantiles
quantile_norm(adata_icam, n_quantiles=100)
quantile_norm(adata_tcga, n_quantiles=100)

In [25]:
# Quantile normalize by batch_key (dataset in this case)
adata_dict = {
    dataset: adata_sc[adata_sc.obs["dataset"] == dataset]
    for dataset in adata_sc.obs["dataset"].unique()
}

# For the class-specific approach could make new label with dataset/platform + cell_type -> Not sure what a min input number should be for quantile norm
batch_key = "dataset"

for dataset in adata_sc.obs["dataset"].unique():
    quantile_norm(
        adata_dict[dataset],
        n_quantiles=(
            100
            if adata_sc[adata_sc.obs[batch_key] == dataset].shape[0] < 1000
            else 1000
        ),
    )

/tmp/ipykernel_277810/1361028706.py:11: ImplicitModificationWarning: Setting element `.layers['quantile_norm']` of view, initializing view as actual.
  adata.layers[key_added] = sklearn.preprocessing.quantile_transform(X, **kwargs)


In [26]:
# Combine back together
adata_sc = anndata.concat(adata_dict, index_unique="-", join="outer", fill_value=0)
adata_sc.obs_names = adata_sc.obs_names.str.rsplit("-", n=1).str[0]

In [27]:
adata_sc.shape

(1846319, 2000)

#### 3a. Quantile normalize (scissor approach)

In [ ]:
# Append bulk to sc data
adata = anndata.concat(
    [adata_sc, adata_icam], index_unique="-", join="outer", fill_value=0
)
adata.obs_names = adata.obs_names.str.rsplit("-", n=1).str[0]

In [ ]:
sh.pp.quantile_norm(adata)

In [ ]:
adata_icam = adata[adata.obs_names.isin(adata_icam.obs_names)].copy()
adata_icam.obs = adata_icam.obs.dropna(axis=1, how="all")
adata_icam.layers["quantile_norm"] = adata_icam.layers["quantile_norm"].toarray()

In [ ]:
adata_sc = adata[adata.obs_names.isin(adata_sc.obs_names)].copy()
adata_sc.obs = adata_sc.obs.dropna(axis=1, how="all")

### 4. Save output

In [28]:
adata_sc.write(f"{artifact_dir}/core_atlas_hvg_2000-adata.h5ad")
adata_tcga.write(f"{artifact_dir}/TCGA_bulk_hvg_2000-adata.h5ad")
adata_icam.write(f"{artifact_dir}/AC_ICAM_bulk_hvg_2000-adata.h5ad")